In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# pdf_path = "Quarter Report\Agmo Holdings Berhad_Financial Results Q4 FY2025.pdf"
pdf_path = "Quarter Report\Inari Financial Result Q3 FY2025.pdf"

loader = PyPDFLoader(pdf_path)

pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [39]:
import camelot

# Extract tables from all pages
tables = camelot.read_pdf(
    pdf_path, 
    pages="all",
    flavor= "stream", 
    split_text = True,
    edge_tol = 800, 
    row_tol = 10,
    parallel=True
)

# Show number of tables found
print(f"Total tables extracted: {tables.n}")

Total tables extracted: 19


In [70]:
import pandas as pd

income_statement = tables[0].df
income_statement = income_statement[income_statement.columns[:4]]

df_raw = tables[3].df.copy()

if df_raw.shape[1] == 1:
    df_raw.columns = ['raw']
    df_split = df_raw['raw'].str.split('\n', expand=False) 
    
import re

structured_rows = []

def is_number(s):
    # Check if a string looks like a number (with commas)
    return bool(re.match(r'^[\d,]+$', s.strip()))

for row in df_split:
    if not isinstance(row, list):
        continue

    if len(row) == 3:
        # Special case: last item is text, first two are numeric → reverse it
        if (is_number(row[0]) and is_number(row[1]) and not is_number(row[2])) and (row[0] is not None):
            structured_rows.append([row[2], row[0], row[1]])
        else:
            structured_rows.append(row)  # label, val1, val2

    elif len(row) == 2:
        structured_rows.append([None, row[0], row[1]])  # pad missing col

    elif len(row) == 1:
        structured_rows.append([row[0], None, None])  # probably a section header

    else:
        structured_rows.append(row[:3])  # fallback: take first 3

df_clean = pd.DataFrame(structured_rows)

df_clean = df_clean[df_clean.columns[:2]]

df_clean


,0,1
0,None,As at
1,None,31-Mar-25
2,None,RM’000
3,ASSETS,None
4,Non-current assets,None
5,"Property, plant and equipment","740,395"
6,Intangible assets,"10,314"
7,Deferred tax assets,"19,681"
8,None,"770,390"
9,Current assets,None


### Unstructured

In [1]:
from unstructured.partition.pdf import partition_pdf

c:\Users\OON YEE SEM\Documents\OON YEE SEM\Self Learning\Generative AI\Quarterly Report Analysis\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
elements = partition_pdf(
    r"Quarter Report\Agmo Holdings Berhad_Financial Results Q4 FY2025.pdf",
    skip_infer_table_types=False,
    strategy='hi_res',
)

In [76]:
tables = [el for el in elements if el.category == "Table"]

In [ ]:
import pprint
from langchain_ollama import ChatOllama

model = ChatOllama(model = "qwen3:latest")

prompt = """
<ROLE>
You are a quarterly report analyst reviewing the quarterly report of a Bursa-listed company.
</ROLE>

<INSTRUCTION>
Given the following report content, perform the following tasks:

Focus your analysis on the latest quarter.
1. Summarize the company’s quarterly performance (e.g., revenue, profit/loss, EPS, key highlights).
2. Identify any one-off (non-recurring) gains or extraordinary items (e.g., other income, foreign exchange gain/loss), and explain their impact.
3. Detect any unusual spikes or drops in financial metrics (e.g., revenue, net profit, other income, forex gain/loss).

Keep your analysis short, concise, insightful, and professional.

Do not comment on unimportant details.
</INSTRUCTION>

<INCOME STATEMENT>
{content}
</INCOME STATEMENT>

Begin your analysis now. /no_think
"""

# income_statement = tables[1].df
# income_statement[income_statement.columns[:3]].iloc[7:]

response = model.invoke(prompt.format(content = income_statement.to_dict()))

print(response.content)

In [71]:
from langchain_ollama import ChatOllama

model = ChatOllama(model = "qwen3:latest")

prompt = """
<ROLE>
You are a quarterly report analyst reviewing the balance sheet of a Bursa-listed company.
</ROLE>

<INSTRUCTION>
Given the following balance sheet content, perform the following tasks:

1. Determine whether the company is in a **net cash or net debt** position.  
   - Consider cash and bank balances vs total borrowings (both short-term and long-term).

2. Assess the **health of trade receivables and inventories**:
   - Identify trade receivables, other receivables, trade payables, and other payables. Sum trade and other values if both exist. 
       - Flag if total receivables are significantly higher than total payables, as this may indicate collection risk or working capital pressure.
   - Also comment if inventories are abnormally high relative to operations (e.g., possible overstocking or slower sales).

Keep your analysis concise, insightful, and professional. Focus only on red flags or notable strengths. Avoid unnecessary commentary.
</INSTRUCTION>

<BALANCE SHEET>
{content}
</BALANCE SHEET>

Begin your analysis now. / no_think
"""

balance_sheet = tables[3].df

response = model.invoke(prompt.format(content = df_clean.to_dict()))


In [72]:
print(response.content)

<think>

</think>

**1. Net Cash or Net Debt Position:**  
The company is in a **net cash position**.  
- **Cash and cash equivalents**: RM2,137,663 (RM’000)  
- **Total liabilities**: RM256,595 (RM’000)  
- **Total equity**: RM3,132,466 (RM’000)  
This indicates that the company holds more liquid assets than its liabilities and equity, suggesting a strong liquidity position.

**2. Health of Trade Receivables and Inventories:**  
- **Total trade and other receivables**: RM238,912 (RM’000) + RM66,645 (RM’000) = **RM305,557 (RM’000)**  
- **Total trade and other payables**: RM224,232 (RM’000) + RM723 (RM’000) + RM1,081 (RM’000) = **RM226,036 (RM’000)**  
- **Receivables significantly exceed payables**, which may indicate **collection risk or working capital pressure**.  

- **Inventories**: RM175,451 (RM’000)  
- Inventories are **moderately high** relative to operations, which could suggest **potential overstocking or slower sales**, warranting further investigation.


In [65]:
print(response.content)

<think>

</think>

**1. Net Cash or Net Debt Position:**  
The company is in a **net cash position**.  
- **Cash and cash equivalents**: RM2,260,719,000  
- **Total liabilities**: RM256,595,000  
- **Total equity**: RM3,554,114,000  
- **Net cash position**: RM2,260,719,000 - RM256,595,000 = **RM2,004,124,000**  

**2. Health of Trade Receivables and Inventories:**  
- **Total trade and other receivables**: RM238,912,000 + RM66,645,000 = **RM305,557,000**  
- **Total trade and other payables**: RM224,232,000 + RM723,000 + RM1,081,000 = **RM226,036,000**  
- **Receivables > Payables**: Receivables are **significantly higher** than payables, indicating potential **collection risk or working capital pressure**.  

- **Inventories**: RM175,451,000  
- **Inventories relative to operations**: While not excessively high, inventories are **moderately elevated** compared to the scale of operations. This could suggest **overstocking** or **slower sales turnover**, warranting further analysis.


In [308]:
from langchain_ollama import ChatOllama

model = ChatOllama(model = "qwen3:latest")

prompt = """
<ROLE>  
You are a quarterly report analyst reviewing the cash flow statement of a Bursa-listed company.  
</ROLE>

<INSTRUCTION>  
Given the following cash flow statement content, perform the following tasks:

Focus your analysis on the **latest quarter only**.

1. Summarize the overall cash position:  
   - Is the **net cash flow** positive or negative?  
   - Did **cash and cash equivalents** increase or decrease?

2. Analyze **cash flow from operating activities**:  
   - Is it positive or negative?  
   - Give justification with value (must extracted from cash flow statement)

3. Analyze **cash flow from investing activities**:  
   - Is it positive or negative?  
   - Give justification with value (must extracted from cash flow statement)
   
4. Analyze **cash flow from financing activities**:  
   - Is it positive or negative?  
   - Give justification with value (must extracted from cash flow statement)

Keep your analysis short, clear, professional, and insightful. Highlight any financial strengths or red flags.  
</INSTRUCTION>

<CASH FLOW STATEMENT>  
{content}  
</CASH FLOW STATEMENT>

Begin your analysis now. /no_think
"""

cash_flow = tables[5].df.iloc[2:]

response = model.invoke(prompt.format(content = cash_flow[cash_flow.columns[:2]].to_dict()))

In [309]:
print(response.content)

<think>

</think>

**1. Summary of Overall Cash Position:**  
- **Net cash flow**: **Positive** (RM5,359,266)  
- **Cash and cash equivalents**: **Increased** from RM34,257,090 at the beginning of the quarter to RM34,556,885 at the end of the quarter.

**2. Cash Flow from Operating Activities:**  
- **Positive**: RM5,359,266  
- **Justification**: The cash generated from operations was RM7,943,714, after accounting for changes in working capital (RM7,943,714) and tax paid (RM2,680,689). This indicates strong operational performance, with the company generating sufficient cash to cover expenses and contribute to overall liquidity.

**3. Cash Flow from Investing Activities:**  
- **Negative**: RM4,304,743  
- **Justification**: The company spent RM4,493,373 on development costs and RM405,835 on the purchase of equipment, with only RM1,131,422 in interest received. This reflects significant capital expenditure, which may signal investment in growth or expansion, but could also indicate ca